In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("FinalCoursework") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import Row

# Helper function to test the correctness of the solutions
def test(var, val, msg=""):
    print("1 test passed.") if var == val else print("1 test failed. " + msg)

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("created_at", StringType(), True),
    StructField("tweet_id", DoubleType(), True),
    StructField("tweet", StringType(), True),
    StructField("likes", IntegerType(), True),
    StructField("retweet_count", IntegerType(), True),
    StructField("source", StringType(), True),
    StructField("user_id", DoubleType(), True),
    StructField("user_name", StringType(), True),
    StructField("user_screen_name", StringType(), True),
    StructField("user_description", StringType(), True),
    StructField("user_join_date", StringType(), True),
    StructField("user_followers_count", IntegerType(), True),
    StructField("user_location", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("long", DoubleType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("continent", StringType(), True),
    StructField("state", StringType(), True),
    StructField("state_code", StringType(), True),
    StructField("collected_at", StringType(), True)
    # Add more StructFields as needed for your dataset
])

In [17]:
# <FILL-IN WITH YOUR CODE>
df_trump = spark.read.option("multiline", True).csv("data/hashtag_donaldtrump.csv", schema=schema, header=True)
df_biden = spark.read.option("multiline", True).csv("data/hashtag_joebiden.csv", schema=schema, header=True)
# Cache your DataFrame
df_trump.cache()
df_biden.cache()
# <FILL-IN WITH YOUR CODE>

DataFrame[created_at: string, tweet_id: double, tweet: string, likes: int, retweet_count: int, source: string, user_id: double, user_name: string, user_screen_name: string, user_description: string, user_join_date: string, user_followers_count: int, user_location: string, lat: double, long: double, city: string, country: string, continent: string, state: string, state_code: string, collected_at: string]

In [4]:
df_trump.count()

2310698

In [5]:
df_biden.count()

1812041

In [18]:
df_trump.show(5)

+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+------------+----------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|            tweet_id|               tweet|likes|retweet_count|            source|             user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|       lat|        long|      city|             country|    continent|               state|state_code|        collected_at|
+-------------------+--------------------+--------------------+-----+-------------+------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+----------+---------